In [1]:
from database import get_thread_sample, get_comments_sample, bulk_update_comments

In [2]:
import flair
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')

2021-05-26 17:16:14,145 loading file C:\Users\Gabry\.flair\models\sentiment-en-mix-distillbert_4.pt


In [2]:
import pandas as pd

In [3]:
def from_cursor_to_df(cursor):
    return pd.DataFrame.from_dict(list(cursor))

In [5]:
thread_field_list = ['selftext', 'subreddit', 'score', 'is_esl_related', 'title', 'created_utc']
comments_field_list = ['body', 'subreddit', 'score', 'is_esl_related', 'created_utc']

In [6]:
#get_xxx_sample returns a mongo cursor with results from the specified aggregation
#
#limit -> use limit to define the number of results (random sample, default = 1000)
#esl_related -> true or false to get only results related or not to the esl. None to get both (default = None)
#include -> list of included fields. None -> return all the fields. default = ['body','_id']

df_comments = from_cursor_to_df(get_comments_sample(limit=10, esl_related = None, include = comments_field_list))
df_thread = from_cursor_to_df(get_thread_sample(limit=10, esl_related = None, include = thread_field_list))

In [7]:
df_comments['date_created'] = pd.to_datetime(df_comments['created_utc'], unit='s')
df_comments.drop(columns=['created_utc'], inplace=True)
df_thread['date_created'] = pd.to_datetime(df_thread['created_utc'], unit='s')
df_thread.drop(columns=['created_utc'], inplace=True)
df_comments['is_esl_related'].fillna(False, inplace=True)
df_thread['is_esl_related'].fillna(False, inplace=True)

In [8]:
df_comments

,_id,body,score,subreddit,is_esl_related,date_created
0,6085ef97c17c650b4e0e2835,Yes. They have fast players.,18,soccer,False,2021-04-17 19:29:38
1,60865d55c17c650b4e132555,Haha. This farce just gets worse. Relegate all...,1,soccer,True,2021-04-20 11:56:56
2,60865a9bc17c650b4e12f352,"""why can't they cater to my exact needs?""",1,soccer,True,2021-04-20 13:52:26
3,60867604c17c650b4e14cb52,I have a college class about three minutes awa...,3,Gunners,False,2021-04-21 10:07:55
4,6086c66dc17c650b4e19e8bf,Wrong place to ask this. Most on this sub thin...,0,coys,False,2021-04-25 20:31:08
5,6086045ec17c650b4e0eb7f1,Hes not going to shit talk when his employers ...,1,soccer,False,2021-04-18 16:27:15
6,6086a4aec17c650b4e17c18a,I keep thinking about how many frames per seco...,1,Gunners,False,2021-04-23 20:21:42
7,6086b202c17c650b4e18b179,Well they have oil 😂😂😂\nDon't fuck with us💪😘👊,17,soccer,False,2021-04-24 19:45:16
8,60865e1bc17c650b4e13321c,"Right so no Bruno, no Keane, no Cantona, no RV...",1,reddevils,True,2021-04-20 12:51:42
9,60865d9fc17c650b4e132ad9,\n&gt;Because there's also the mass infrastruc...,1,soccer,True,2021-04-20 17:15:00


In [9]:
def sentiment(x): 
    s = flair.data.Sentence(x)
    flair_sentiment.predict(s)
    return s.labels

In [10]:
df_comments['Sentiment'] = df_comments['body'].apply(
    lambda x: sentiment(x)
)

In [11]:
str(df_comments['Sentiment'][0][0]).split()

['POSITIVE', '(0.9966)']

In [12]:
df_comments['Sentiment_direction'] = df_comments['Sentiment'].apply(
    lambda x: str(x[0]).split()[0]
)
df_comments['Sentiment_accuracy'] = df_comments['Sentiment'].apply(
    lambda x: float(str(x[0]).split()[1].replace('(', '').replace(')', '') )
)